<h1>Amazon apparel Recommendations</h1>

### I am SOUHARDYA ADHIKARY
You can see my profile at [GITHUB](https://github.com/SOUHARDYAADHIKARY1999)

### Connect me at my linkedIn profile
https://www.linkedin.com/in/souhardya-adhikary-a10a10156/


### [1.0] Import the packages

In [1]:
import pandas as pd
from collections import Counter

In [2]:
# we have give a json file which consists of all information about
# the products
# loading the data using pandas' read_json file.
data=pd.read_json(open('/Users\DELL\PythonProjects\RECOMENDATION_ENGINE/tops_fashion.json',"r",encoding="utf-8"))

In [3]:
print(data)

         sku        asin product_type_name formatted_price author  \
0       None  B016I2TS4W             SHIRT            None   None   
1       None  B01N49AI08             SHIRT            None   None   
2       None  B01JDPCOHO             SHIRT            None   None   
3       None  B01N19U5H5             SHIRT            None   None   
4       None  B004GSI2OS             SHIRT          $26.26   None   
...      ...         ...               ...             ...    ...   
183133  None  B01MSALTSO         OUTERWEAR          $14.58   None   
183134  None  B015W98YQK             SHIRT            None   None   
183135  None  B075756PGC             SHIRT            None   None   
183136  None  B074L8FVTT             SHIRT          $44.99   None   
183137  None  B074FTYJQC             SHIRT            None   None   

                    color          brand      publisher  \
0                    None          FNC7C           None   
1                    None   FIG Clothing           No

In [4]:
print ('Number of data points : ', data.shape[0])
print ('Number of features/variables:', data.shape[1])

Number of data points :  183138
Number of features/variables: 19


### Terminology
Let's see the attributes we have 
### Each column name is a specific feature of the items

In [5]:
# each product/item has 19 features in the raw dataset.
print(data.columns)# prints column-names or feature-names.

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')


Of these 19 features, I will be using only 6 features in this workshop.
    1. asin  ( Amazon standard identification number)
    2. brand ( brand to which the product belongs to )
    3. color ( Color information of apparel, it can contain many colors as   a value ex: red and black stripes ) 
    4. product_type_name (type of the apperal, ex: SHIRT/TSHIRT )
    5. medium_image_url  ( url of the image )
    6. title (title of the product.)
    7. formatted_price (price of the product)

In [6]:
data = data[['asin', 'brand', 'color', 'medium_image_url', 'product_type_name', 'title', 'formatted_price']]

In [7]:
print(data)

              asin          brand              color  \
0       B016I2TS4W          FNC7C               None   
1       B01N49AI08   FIG Clothing               None   
2       B01JDPCOHO   FIG Clothing               None   
3       B01N19U5H5        Focal18               None   
4       B004GSI2OS    FeatherLite  Onyx Black/ Stone   
...            ...            ...                ...   
183133  B01MSALTSO      TOOGOO(R)              Black   
183134  B015W98YQK     VOGUE CODE   Monochrome Plaid   
183135  B075756PGC       Wrangler               Pink   
183136  B074L8FVTT  susana monaco               Rose   
183137  B074FTYJQC       Sexybaby               None   

                                         medium_image_url product_type_name  \
0       https://images-na.ssl-images-amazon.com/images...             SHIRT   
1       https://images-na.ssl-images-amazon.com/images...             SHIRT   
2       https://images-na.ssl-images-amazon.com/images...             SHIRT   
3       htt

In [8]:
print ('Number of data points : ', data.shape[0])
print ('Number of features/variables:', data.shape[1])

Number of data points :  183138
Number of features/variables: 7


In [9]:
data.head

<bound method NDFrame.head of               asin          brand              color  \
0       B016I2TS4W          FNC7C               None   
1       B01N49AI08   FIG Clothing               None   
2       B01JDPCOHO   FIG Clothing               None   
3       B01N19U5H5        Focal18               None   
4       B004GSI2OS    FeatherLite  Onyx Black/ Stone   
...            ...            ...                ...   
183133  B01MSALTSO      TOOGOO(R)              Black   
183134  B015W98YQK     VOGUE CODE   Monochrome Plaid   
183135  B075756PGC       Wrangler               Pink   
183136  B074L8FVTT  susana monaco               Rose   
183137  B074FTYJQC       Sexybaby               None   

                                         medium_image_url product_type_name  \
0       https://images-na.ssl-images-amazon.com/images...             SHIRT   
1       https://images-na.ssl-images-amazon.com/images...             SHIRT   
2       https://images-na.ssl-images-amazon.com/images...   

### Missing data for various features.

####  Basic stats for the feature: product_type_name

In [10]:
# We have total 72 unique type of product_type_names
print(data['product_type_name'].describe())

# 91.62% (167794/183138) of the products are shirts,

count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object


In [11]:
# We have total 10577 unique type of brands
print(data['brand'].describe())

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object


In [12]:
# So we also have simiral types of products 
print(data['title'].describe())

count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object


In [13]:
# names of different product types
print(data['product_type_name'].unique())

['SHIRT' 'SWEATER' 'APPAREL' 'OUTDOOR_RECREATION_PRODUCT'
 'BOOKS_1973_AND_LATER' 'PANTS' 'HAT' 'SPORTING_GOODS' 'DRESS' 'UNDERWEAR'
 'SKIRT' 'OUTERWEAR' 'BRA' 'ACCESSORY' 'ART_SUPPLIES' 'SLEEPWEAR'
 'ORCA_SHIRT' 'HANDBAG' 'PET_SUPPLIES' 'SHOES' 'KITCHEN' 'ADULT_COSTUME'
 'HOME_BED_AND_BATH' 'MISC_OTHER' 'BLAZER' 'HEALTH_PERSONAL_CARE'
 'TOYS_AND_GAMES' 'SWIMWEAR' 'CONSUMER_ELECTRONICS' 'SHORTS' 'HOME'
 'AUTO_PART' 'OFFICE_PRODUCTS' 'ETHNIC_WEAR' 'BEAUTY'
 'INSTRUMENT_PARTS_AND_ACCESSORIES' 'POWERSPORTS_PROTECTIVE_GEAR' 'SHIRTS'
 'ABIS_APPAREL' 'AUTO_ACCESSORY' 'NONAPPARELMISC' 'TOOLS' 'BABY_PRODUCT'
 'SOCKSHOSIERY' 'POWERSPORTS_RIDING_SHIRT' 'EYEWEAR' 'SUIT'
 'OUTDOOR_LIVING' 'POWERSPORTS_RIDING_JACKET' 'HARDWARE' 'SAFETY_SUPPLY'
 'ABIS_DVD' 'VIDEO_DVD' 'GOLF_CLUB' 'MUSIC_POPULAR_VINYL'
 'HOME_FURNITURE_AND_DECOR' 'TABLET_COMPUTER' 'GUILD_ACCESSORIES'
 'ABIS_SPORTS' 'ART_AND_CRAFT_SUPPLY' 'BAG' 'MECHANICAL_COMPONENTS'
 'SOUND_AND_RECORDING_EQUIPMENT' 'COMPUTER_COMPONENT' 'JEWELRY'
 'B

In [14]:
# This is the type of data set we have
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [15]:
print(data)

              asin          brand              color  \
0       B016I2TS4W          FNC7C               None   
1       B01N49AI08   FIG Clothing               None   
2       B01JDPCOHO   FIG Clothing               None   
3       B01N19U5H5        Focal18               None   
4       B004GSI2OS    FeatherLite  Onyx Black/ Stone   
...            ...            ...                ...   
183133  B01MSALTSO      TOOGOO(R)              Black   
183134  B015W98YQK     VOGUE CODE   Monochrome Plaid   
183135  B075756PGC       Wrangler               Pink   
183136  B074L8FVTT  susana monaco               Rose   
183137  B074FTYJQC       Sexybaby               None   

                                         medium_image_url product_type_name  \
0       https://images-na.ssl-images-amazon.com/images...             SHIRT   
1       https://images-na.ssl-images-amazon.com/images...             SHIRT   
2       https://images-na.ssl-images-amazon.com/images...             SHIRT   
3       htt

In [16]:
product_type_count = Counter(list(data['product_type_name']))
product_type_count

Counter({'SHIRT': 167794,
         'SWEATER': 837,
         'APPAREL': 3549,
         'OUTDOOR_RECREATION_PRODUCT': 729,
         'BOOKS_1973_AND_LATER': 3336,
         'PANTS': 363,
         'HAT': 18,
         'SPORTING_GOODS': 1281,
         'DRESS': 1584,
         'UNDERWEAR': 425,
         'SKIRT': 107,
         'OUTERWEAR': 796,
         'BRA': 95,
         'ACCESSORY': 636,
         'ART_SUPPLIES': 17,
         'SLEEPWEAR': 142,
         'ORCA_SHIRT': 290,
         'HANDBAG': 21,
         'PET_SUPPLIES': 2,
         'SHOES': 198,
         'KITCHEN': 46,
         'ADULT_COSTUME': 25,
         'HOME_BED_AND_BATH': 11,
         'MISC_OTHER': 62,
         'BLAZER': 249,
         'HEALTH_PERSONAL_CARE': 92,
         'TOYS_AND_GAMES': 80,
         'SWIMWEAR': 41,
         'CONSUMER_ELECTRONICS': 1,
         'SHORTS': 14,
         'HOME': 34,
         'AUTO_PART': 3,
         'OFFICE_PRODUCTS': 7,
         'ETHNIC_WEAR': 34,
         'BEAUTY': 21,
         'INSTRUMENT_PARTS_AND_ACCESSO

In [17]:
# Let's see the most common product type we have
product_type_count.most_common(1)

[('SHIRT', 167794)]

In [18]:
# Let's explore the other products we have
product_type_count.most_common(10)

[('SHIRT', 167794),
 ('APPAREL', 3549),
 ('BOOKS_1973_AND_LATER', 3336),
 ('DRESS', 1584),
 ('SPORTING_GOODS', 1281),
 ('SWEATER', 837),
 ('OUTERWEAR', 796),
 ('OUTDOOR_RECREATION_PRODUCT', 729),
 ('ACCESSORY', 636),
 ('UNDERWEAR', 425)]

####  Basic stats for the feature: brand

In [19]:
# there are 10577 unique brands
print(data['brand'].describe())

# 183138 - 182987 = 151 missing values.

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object


In [20]:
brand_count = Counter(list(data['brand']))
brand_count.most_common(10)

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

####  Basic stats for the feature: color

In [21]:

print(data['color'].describe())


# we have 7380 unique colors
# 7.2% of products are black in color
# 64956 of 183138 products have brand information. That's approx 35.4%.

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object


In [22]:
color_count = Counter(list(data['color']))
color_count.most_common(10)

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

####  Basic stats for the feature: formatted_price

In [23]:
 
print(data['formatted_price'].describe())

# Only 28,395 (15.5% of whole data) products with price information

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object


In [24]:
price_count = Counter(list(data['formatted_price']))
price_count.most_common(10)

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

#### Basic stats for the feature: title


In [25]:
print(data['title'].describe())

# All of the products have a title. 
# Titles are fairly descriptive of what the product is. 
# We use titles extensively in this workshop 
# as they are short and informative.


count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object


## We save data files at every major step in our processing in "pickle" files. If you are stuck anywhere (or) if some code takes too long to run on your laptop, you may use the pickle files we give you to speed things up.

In [26]:
data.to_pickle('pickels/180k_apparel_data')

In [27]:
# consider products which have price information
# data['formatted_price'].isnull() => gives the information 
#about the dataframe row's which have null values price == None|Null
data = data.loc[~data['formatted_price'].isnull()]
print('Number of data points After eliminating price=NULL :', data.shape[0])

Number of data points After eliminating price=NULL : 28395
